In [1]:
import numpy as np, scipy.sparse as sp

A = np.matrix(np.zeros((5,10)))
B = np.zeros((10,1))
print ((A*B).shape)
print (np.dot(A,B))
print (np.dot(A,B).shape)

A = sp.csr_matrix(np.zeros((5,10)), dtype=np.float32)
B = sp.csc_matrix((10,1), dtype=np.float32)
print(A*B)
print ((A*B).shape)
print(A.dot(B))
print(A.dot(B).shape)
#print (np.dot(A,B).shape)

A = sp.csr_matrix(np.zeros((5,10)))
B = np.zeros((10,1))
print ((A*B).shape)
#print (np.dot(A,B).shape)

(5, 1)
[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]
(5, 1)

(5, 1)

(5, 1)
(5, 1)


In [2]:
import numpy, scipy.sparse
n = 100000
x = (numpy.random.rand(n) * 2).astype(int).astype(float) # 50% sparse vector
x_csr = scipy.sparse.csr_matrix(x)
x_dok = scipy.sparse.dok_matrix(x.reshape(x_csr.shape))

In [3]:
%time numpy.dot(x,x)

CPU times: user 28 ms, sys: 0 ns, total: 28 ms
Wall time: 20.3 ms


49954.0

In [4]:
%time x.dot(x)

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 3.67 ms


49954.0

In [5]:
%time x_dok * x_dok.T

CPU times: user 2.77 s, sys: 16 ms, total: 2.78 s
Wall time: 2.12 s


<1x1 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [6]:
%time x_csr.multiply(x_csr).sum()

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 1.8 ms


49954.0

In [7]:
%time x_csr * x_csr.T

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 2.78 ms


<1x1 sparse matrix of type '<class 'numpy.float64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [17]:
import theano

theano.config.device

'cpu'

In [8]:
import numpy as np
import pandas as pd
import scipy
from scipy.sparse import csr_matrix, diags, eye
from sklearn.cluster import SpectralClustering
from matplotlib import pyplot as plt
import pylab

plt.style.use('ggplot')
pylab.rcParams['figure.figsize'] = (16.0, 10.0)

%matplotlib inline

In [9]:
#file = "../dataset/facebook_combined.txt.gz"
file = "../dataset/twitter_combined.txt.gz"

df = pd.read_csv(file, compression='infer', sep=" ", header=None, names=["a", "b"])

nodes = list(set(df.a) | set(df.b))
nodes.sort()
print(nodes[:20])
nodes_to_idx = {n:i for i,n in enumerate(nodes)}

df = pd.DataFrame({"a": df.a.apply(lambda x: nodes_to_idx[x]), 
                   "b": df.b.apply(lambda x: nodes_to_idx[x])})

e = len(df)
v = max(np.max(np.unique(df["a"])), np.max(np.unique(df["b"]))) + 1
print(e,v)

print(df.head())

facebook2 = csr_matrix((np.ones(e), (df.a, df.b)), shape=(v,v), dtype=np.float32)
facebook2

[12, 13, 14, 17, 20, 47, 66, 107, 190, 224, 246, 257, 265, 291, 295, 324, 380, 409, 414, 418]
2420766 81306
       a      b
0  64848  33444
1  15266  30189
2  76829  19302
3  65515  58679
4  52420  16766


<81306x81306 sparse matrix of type '<class 'numpy.float32'>'
	with 1768149 stored elements in Compressed Sparse Row format>

In [10]:
import theano
import numpy as np
import scipy.sparse as sp
import theano.tensor as T
from theano import sparse

In [11]:
x = sparse.csc_matrix(name='x', dtype='float32')
y = sparse.add(x, x)
s = theano.function([x], y)
z = sparse.dot(x, x)
p = theano.function([x], y)

In [12]:
%time facebook3_np = facebook2 + facebook2
%time facebook3 = s(facebook2)

CPU times: user 12 ms, sys: 4 ms, total: 16 ms
Wall time: 16.5 ms
CPU times: user 64 ms, sys: 4 ms, total: 68 ms
Wall time: 65.9 ms


In [13]:
# facebook2.tocsc()
# %time facebook2.multiply(facebook2)

In [20]:
%time facebook4_np = facebook2.dot(facebook2.tocsc())
%time facebook4 = p(facebook2)
%time facebook4 = p(facebook2)

CPU times: user 2.65 s, sys: 36 ms, total: 2.69 s
Wall time: 2.7 s
CPU times: user 52 ms, sys: 0 ns, total: 52 ms
Wall time: 53.1 ms
CPU times: user 48 ms, sys: 0 ns, total: 48 ms
Wall time: 48.5 ms


In [22]:
print (theano.config.device)
%time facebook5 = p(p(facebook2))
facebook5

cpu
CPU times: user 48 ms, sys: 4 ms, total: 52 ms
Wall time: 53.8 ms


<81306x81306 sparse matrix of type '<class 'numpy.float32'>'
	with 1768149 stored elements in Compressed Sparse Column format>